In [ ]:
ACCESS_TOKEN = "1213270339293077504-0VGZgQTrbUs3cNfITH8SkytxGuLmcE"
ACCESS_TOKEN_SECRET = "zPqKpQumQkcKf52iIakDtjH6C6kbFN6M4w2H4im1yZ48a"
CONSUMER_KEY = "GgMgpR0X0Bf3j6Y8UzeBY5eCp"
CONSUMER_SECRET = "hA13Bslmb4Bi9Vo4WGwRKSVELiksYmGGOiH7lITvWnarS4HOMn"

In [ ]:
import tweepy
from datetime import datetime
import csv
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import json
import pandas as pd
import numpy as np

keywords = ['#Bernie', '#Trump', '#petebuttigieg']
#keywords = ['#Democrats2020','#]
filename = 'C:\\Users\\abhin\\Downloads\\twitter_tweets.json'
end_date = datetime.strptime("2020-02-16", '%Y-%m-%d').date()
flag = False
twitter_data = {}

class stdOutListener(StreamListener):
    
    def __init__(self, filename):
        self.tweet_count = 0
        
    def on_data(self, stream):
    
        if(self.tweet_count > 10000):
            return False
    
        data = json.loads(stream)
        tweet_date = datetime.strptime(data['created_at'], '%a %b %d %H:%M:%S %z %Y').date()
        
        #if 'retweeted_status' in [*data]:
            #print("Retweet found, ignoring!")
            #print(data['retweeted_status']['retweet_count'])
            #return
            
        if data['lang'] != 'en':
            print("Non English Language, ignoring!")
            return
            
        if(tweet_date < end_date):
            return False
        
        dict_ = {'created_at':str(),  # time and date
                 'text':str(),  # of the tweet
                 'name' : str(), # of the account
                 'location' : str(), 
                 'description' : str(), # of the profile of the user
                 'followers_count': int(), # number of followers of the user
                 'friends_count': int(), # number of friends of the user
                 'quote_count':int(), # times this tweet has been quoted
                 'reply_count': int(), # times this tweet has been replied to
                 'retweet_count':int(), # times this tweet has been retweeted 
                 'favorite_count':int() # times this tweet has been favourited
                }
        
        if 'retweeted_status' in [*data]:
            
            dict_['created_at'] = data['retweeted_status']['created_at']
            dict_['text'] = data['retweeted_status']['text']

            dict_['name'] = data['retweeted_status']['user']['name']
            dict_['location'] = data['retweeted_status']['user']['location']
            dict_['description'] = data['retweeted_status']['user']['description']
            dict_['followers_count'] = data['retweeted_status']['user']['followers_count']
            dict_['friends_count'] = data['retweeted_status']['user']['friends_count']

            dict_['quote_count'] = data['retweeted_status']['quote_count']
            dict_['reply_count'] = data['retweeted_status']['reply_count']
            dict_['retweet_count'] = data['retweeted_status']['retweet_count']
            dict_['favorite_count'] = data['retweeted_status']['favorite_count']
            
        else:
            
            dict_['created_at'] = data['created_at']
            dict_['text'] = data['text']

            dict_['name'] = data['user']['name']
            dict_['location'] = data['user']['location']
            dict_['description'] = data['user']['description']
            dict_['followers_count'] = data['user']['followers_count']
            dict_['friends_count'] = data['user']['friends_count']

            dict_['quote_count'] = data['quote_count']
            dict_['reply_count'] = data['reply_count']
            dict_['retweet_count'] = data['retweet_count']
            dict_['favorite_count'] = data['favorite_count']
        
        with open('data.csv', 'a') as f:
            w = csv.DictWriter(f, dict_.keys())
            w.writerow(dict_)
            self.tweet_count += 1
            
            print("Added new tweet to file!")
            
    def on_error(self, status_code):
        if status_code == 420:
            return False
        
class twitterStreamer():
    
    def setConnection(self, filename, keywords):
        try:
            listener = stdOutListener(filename)
            auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
            auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
            print("Connection has been established successfully!")
            return(auth, listener)
        
        except:
            print("Connection not established. Try Again Later.")
            
    def getTweets(self, auth, listener):
        stream = Stream(auth, listener)
        stream.filter(track=keywords)
    
if __name__ == "__main__":
    tweets = twitterStreamer()
    auth, listener = tweets.setConnection(filename, keywords) 
    tweets.getTweets(auth, listener)
    print("HERE!!!!!!")